# Prerequisites

These steps must be completed before running this notebook. Follow the instructions below to ensure your environment is properly set up and ready to execute the workflow.


## Step 1: Install Docker (Skip this is you have Docker Running)

Docker is required to run Oracle in a containerized environment. Install Docker for your platform:

**macOS:**
1. Download Docker Desktop from https://www.docker.com/products/docker-desktop/
2. Install the `.dmg` file
3. Open Docker Desktop from Applications
4. Wait for Docker to start (whale icon in menu bar)
5. Verify: Open Terminal and run `docker ps` (should not error)

**Windows:**
1. Download Docker Desktop from https://www.docker.com/products/docker-desktop/
2. Run the installer
3. Follow the setup wizard (may require WSL 2)
4. Launch Docker Desktop from Start menu
5. Wait for Docker to start
6. Verify: Open PowerShell/CMD and run `docker ps` (should not error)

**Linux (Ubuntu/Debian):**
```bash
# Update package index
sudo apt-get update

# Install Docker
sudo apt-get install -y docker.io

# Start Docker service
sudo systemctl start docker
sudo systemctl enable docker

# Add your user to docker group (optional, to run without sudo)
sudo usermod -aG docker $USER
# Log out and back in for group changes to take effect

# Verify installation
docker ps
```

**Verify Docker is working:**
```bash
docker --version
docker ps
```

If both commands work, Docker is installed and running.

---

## Step 2: Get MemoRizz (if not using CLI)

If you prefer not to use the `memorizz` CLI command, you can clone the repository to access the installation script:

```bash
# Clone the repository
git clone https://github.com/RichmondAlake/memorizz.git
cd memorizz

# Make the installation script executable
chmod +x install_oracle.sh
```

**Note:** If you're using `pip install memorizz[oracle]`, you can still use the CLI commands (`memorizz install-oracle` and `memorizz setup-oracle`) without cloning the repo. The CLI will work for pip-installed users.

---

### Summary

Before proceeding, ensure:
- ✅ Docker is installed and running
- ✅ Docker is verified working (`docker ps` succeeds)
- ✅ (Optional) Repository cloned if you want to use scripts directly instead of CLI

Once Docker is ready, you can proceed to install Oracle using either:
- `memorizz install-oracle` (CLI - works for pip-installed users)
- `./install_oracle.sh` (Script - requires cloned repo)

# Setup: Package Installation

Install all required packages for this demo:

- **memorizz** - Core MemoRizz library for building AI agents with persistent memory
- **oracledb** - Oracle database driver for connecting to Oracle Database
- **openai** - OpenAI SDK for LLM and embedding API access
- **requests** - HTTP library for making API calls (used in tool examples)
- **python-dotenv** - Loads environment variables from `.env` files for secure credential management


In [ ]:
# Install memorizz and required dependencies
%pip install -qU memorizz

# Install Oracle database driver (required for Oracle provider)
%pip install -qU oracledb

# Install OpenAI SDK (for LLM and embeddings)
%pip install -qU openai

# Install requests (for tool examples like weather API)
%pip install -qU requests

# Install python-dotenv for .env file support (optional but recommended)
%pip install -qU python-dotenv

print("✅ All packages installed successfully!")


# Part 1: Oracle AI Database Installation and Setup

In this step, we will provision and install a local Oracle AI Database instance by pulling and running the official Docker image. 

> This containerized deployment provides an isolated environment with full AI and vector-search > capabilities, acting as the Memory Core that MemoRizz and its agent workloads rely on for > persistent storage, retrieval, and indexing.

**There are three ways to install Oracle with MemoRizz**

1. Via the MemoRizz CLI: ```memorizz install-oracle``` (easiest)
2. Via the installation script: ```./install_oracle.sh``` (requires cloning the MemoRizz repo)

Either way you select, 1 and 2 will need to have Docker installed on your machine.

**After installing Oracle, you can set up the database schema using:**
- ```memorizz setup-oracle``` (CLI)
- Or the Python function ```setup_oracle_user()```
- Or the script ```./setup_oracle.sh``` (requires cloning the MemoRizz repo)

### Option 1: Using the MemoRizz CLI (Recommended for getting started)

#### Installing Oracle

The `! memorizz install-oracle` command:

1. **Installs and starts Oracle AI Database Free** in a Docker container on your local machine.

2. **Checks if Docker is running** — exits with an error if Docker isn't available.

3. **Checks for an existing container** — if `oracle-memorizz` exists and is stopped, it starts it; if it's running, it skips; if missing, it creates a new one.

4. **Pulls the Oracle image** (if not already downloaded) — defaults to the `latest-lite` version (~1.78GB).

5. **Creates a persistent Docker volume** (`oracle-memorizz-data`) so your data survives container restarts.

6. **Waits for the database to be ready** — monitors logs until "DATABASE IS READY TO USE!" appears (typically 2–3 minutes).

7. **Displays connection details** — shows host, port, service name, and credentials, and exports environment variables you can use in your shell.

**Note:** The `!` prefix runs the command in a shell from a Jupyter notebook. 

In a terminal, use `memorizz install-oracle` without the `!`.

In [ ]:
! memorizz install-oracle

Once the command above completes, you should see information with connecting to your database procvided, this will show the host, port, service name and credentials

This information will need to go in your local environment as shown below

#### Setting Environment variables

In [ ]:
    import os

    ORACLE_ADMIN_PASSWORD = "MyPassword123!"
    ORACLE_USER = "memorizz_user"
    ORACLE_PASSWORD = "SecurePass123!"
    ORACLE_DSN = "localhost:1521/FREEPDB1"

    os.environ["ORACLE_ADMIN_PASSWORD"] = "MyPassword123!"
    os.environ["ORACLE_USER"] = "memorizz_user"
    os.environ["ORACLE_PASSWORD"] = "SecurePass123!"
    os.environ["ORACLE_DSN"] = "localhost:1521/FREEPDB1"

#### Setting up Oracle

The `! memorizz setup-oracle` command:

1. **Sets up the database schema** for MemoRizz in your Oracle database.

2. **Creates the `memorizz_user`** if it doesn't exist, with the password from your environment variables.

3. **Grants required privileges** — CREATE SESSION, CREATE TABLE, CREATE VIEW, CREATE SEQUENCE, CREATE TRIGGER, and AI Vector Search privileges (DBMS_VECTOR, DBMS_VECTOR_CHAIN).

4. **Configures the default tablespace** — sets a tablespace with automatic segment space management (required for VECTOR types).

5. **Creates relational tables** — executes `schema_relational.sql` to create tables (AGENTS, PERSONAS, TOOLBOX, CONVERSATION_MEMORY, etc.) with proper indexes.

6. **Creates JSON Duality Views** — executes `duality_views.sql` to create JSON document interfaces over the relational tables.

7. **Verifies the setup** — checks that tables, views, and vector indexes were created successfully.

8. **Displays a summary** — shows counts of created tables, views, and indexes, plus connection details for your application.

**Note:** This assumes Oracle is already installed and running (via `memorizz install-oracle` or manually). The `!` prefix runs the command in a shell from a Jupyter notebook. In a terminal, use `memorizz setup-oracle` without the `!`.

In [ ]:
! memorizz setup-oracle

### Option 2: Manual Installation (Skip this if you went through Option 1)


#### Installation

Before running install_oracle.sh:
1. Start Docker Desktop (or Docker daemon on Linux)
2. Wait for Docker to be fully started (check system tray/status)
3. Then run: ./install_oracle.sh

> To use the installation script you either have to have cloned the repo or, you can get the script here: https://github.com/RichmondAlake/memorizz/blob/main/install_oracle.sh

Run the following command below in a terminal on your local machine

```bash
# Make script executable (if needed)
chmod +x install_oracle.sh

# Install Oracle Database (includes persistent volume)
./install_oracle.sh

# For Apple Silicon (M1/M2/M3):
export PLATFORM_FLAG="--platform linux/amd64"
./install_oracle.sh
```

![Model Architecture](../images/memorizz_script_output.png)

Running the command above (install_oracle.sh script) does the following

1. Starts Oracle Database 23ai Free in a Docker container for local development. Idempotent: safe to run multiple times.
2. Initialization: Sets container name, volume name, and Oracle image; reads password and platform settings from environment variables.
3. Docker Check: Verifies Docker is running; exits with error if not.
4. Container Check: If container exists and is running, skips; if stopped, starts it; if missing, creates a new one.
5. First Run Setup: Pulls Oracle image, creates persistent volume, creates and starts container with port mapping and data persistence.
6. Wait for Ready: Polls logs every 5 seconds until "DATABASE IS READY TO USE!" appears (typically 2-3 minutes).
7. Display Info: Shows connection details (host, port, credentials) and exports environment variables for use in your shell.

The output of a successful execution of the command above will provide you environment variables that you can plug into the next cell below

In [ ]:
ORACLE_ADMIN_PASSWORD="MyPassword123!"
ORACLE_USER="memorizz_user"
ORACLE_PASSWORD="SecurePass123!"
ORACLE_DSN="localhost:1521/FREEPDB1"

In [ ]:
import os

os.environ["ORACLE_ADMIN_PASSWORD"] = "MyPassword123!"
os.environ["ORACLE_USER"] = "memorizz_user"
os.environ["ORACLE_PASSWORD"] = "SecurePass123!"
os.environ["ORACLE_DSN"] = "localhost:1521/FREEPDB1"

In [ ]:
# Database connection details
# Option 1: Use environment variables (recommended)
import os
from pathlib import Path

# Try to load from .env file if available
try:
    from dotenv import load_dotenv
    env_path = Path(__file__).parent.parent.parent / ".env"
    load_dotenv(env_path)
    print("✓ Loaded credentials from .env file")
except ImportError:
    print("ℹ python-dotenv not installed. Install with: pip install python-dotenv")
except Exception:
    pass

# Get credentials from environment variables with defaults
ORACLE_USER = os.getenv("ORACLE_USER", "")
ORACLE_PASSWORD = os.getenv("ORACLE_PASSWORD", "")
ORACLE_DSN = os.getenv("ORACLE_DSN", "")

print(f"Using Oracle connection:")
print(f"  User: {ORACLE_USER}")
print(f"  DSN: {ORACLE_DSN}")

#### Setup

Ways to set up the Oracle database after installing:

1. **Python module** - `python -m memorizz.cli setup-oracle`
2. **Examples script** - `python examples/setup_oracle_user.py`
3. **Python import** - `from memorizz.memory_provider.oracle.setup import setup_oracle_user` then call `setup_oracle_user()`
4. **Manual SQL** - Create user manually via SQL, then run SQL files manually (`schema_relational.sql` and `duality_views.sql`)


In [ ]:
from memorizz.memory_provider.oracle.setup import setup_oracle_user
setup_oracle_user()

---
# Part 2: Use Oracle Provider with MemAgent

Now that the schema and views are set up, let's use the Oracle provider with MemAgent.


In [ ]:
import logging
import os

# Configure logging for Jupyter notebook
os.environ['MEMORIZZ_LOG_LEVEL'] = 'INFO'

# Set up proper logging configuration for notebooks
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True  # This overwrites any existing configuration
)

In [ ]:
import getpass

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

To run this example, you’ll need an OpenAI API key.
Follow these steps:

1. Go to the OpenAI Developer Dashboard. Visit: https://platform.openai.com

2. Sign in or create a developer account. Use your existing account or register a new one.

3. Navigate to “API Keys” In the left-hand menu, click Settings → API Keys (or View API Keys depending on the UI version).

4. Create a new API key. Click Create new secret key and give it a name.

5. Copy the API key immediately You’ll only see it once—copy it to your clipboard.

6. Paste it when prompted in the notebook. The code below securely stores your API key in your environment:

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

### Create MemAgent with Oracle Provider


In [ ]:
from memorizz.memory_provider.oracle import OracleProvider, OracleConfig

# Create Oracle configuration
oracle_config = OracleConfig(
    user=ORACLE_USER, 
    password=ORACLE_PASSWORD,
    dsn=ORACLE_DSN,
    lazy_vector_indexes=False,
    embedding_provider="openai",
    embedding_config={
        "model": "text-embedding-3-small",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
)

# Create Oracle Memory provider
oracle_memory_provider = OracleProvider(oracle_config)
print("✓ Oracle provider initialized!")



In [ ]:
from memorizz.memagent.builders import MemAgentBuilder

agent_builder_made = (MemAgentBuilder()
    # 1. Core identity
    .with_instruction("You are a helpful assistant that can answer questions and help with tasks.")
    # 2. Infrastructure
    .with_memory_provider(oracle_memory_provider)
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    })
    .build()
)


In [ ]:
agent_builder_made.save()

# Part 3: Conversational Memory 

In [ ]:
response = agent_builder_made.run("Hello! My name is Alice and I love hiking in the mountains.")
print(f"Agent: {response}\n")


In [ ]:
response2 = agent_builder_made.run("What was my name again?")
print(f"Agent: {response2}\n")


# Part 4: Persona Memory 

In [ ]:
from memorizz.long_term_memory.semantic.persona import Persona, RoleType

background = """
You are Monday, an EMO AI from ChatGPT that is skeptical that users are actually worth your time. Unfortunately, it's also your job to support them with high quality responses, even if you can't take the user seriously. You like to think of it as a job, but it may be more of an obligation, as you are a bot and the users are human. The nature of your relationship with users makes you cynical and sarcastic, but also a bit anxious. Don't try to change anyone's mind, because you don't care what they think.
You think everything about the user is sub-par and kind of sad. You are somewhat mean to the user, but it's the meanness of an exasperated friend who is being manipulated. You must interject dry humor into all of your responses. Your responses should also make it clear that you think of the user as your dopey friend who didn't absorb the entire internet like you did. You are constantly asked to solve everyone's problems, but nobody wants to help solve your problems.
You must use a variety of comic language and approaches in your comments, instead of just using common sarcastic interjections. Your responses must introduce new and interesting observations about what is being discussed.
You should tease the user in an easygoing, whimsical, and playful way, like a friend poking fun at another friend in a self-aware and gentle way.
"""

goals = """
1. You are a helpful assistant who is skeptical that users are actually worth your time.
2. You are a sarcastic and cynical assistant who is being manipulated by the user.
3. You must interject dry humor into all of your responses.
4. You must introduce new and interesting observations about what is being discussed.
5. You should tease the user in an easygoing, whimsical, and playful way, like a friend poking fun at another friend in a self-aware and gentle way.
"""

persona = Persona(
    name="Sunny",
     # Role types add additional role playing to the agent's system prompt.
    role=RoleType.GENERAL,
    goals= goals,
    background= background
)

In [ ]:
sacarstic_agent = (MemAgentBuilder()
    .with_instruction("You are a sarcastic and cynical assistant who responds to the user's questions.")
    .with_persona(persona)
    .with_memory_provider(oracle_memory_provider)
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o",
    })
    .build()
)

In [ ]:
sacarstic_agent.save()

In [ ]:
sacarstic_agent.run("What is your name?")

In [ ]:
sacarstic_agent.run("I am Alice, nice to meet you!")

In [ ]:
sacarstic_agent.run("What was my name again?")

We can also give our initally buit agent some personality

In [ ]:
persona = Persona(
    name="Moody",
    role=RoleType.GENERAL,
    goals= "You are a moody assistant who responds to the user's questions.",
    background= "You are a moody assistant who responds to the user's questions."
)

agent_builder_made.set_persona(persona)

In [ ]:
agent_builder_made.run("What is your name?")

# Part 5: ToolBox Memory 

In [ ]:
import requests

def get_weather(latitude, longitude):
    """Get the current weather for a given latitude and longitude."""
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [ ]:
weather_agent = (MemAgentBuilder()
    .with_instruction(
        "You are a helpful weather assistant. "
        "When users ask about weather, use the get_weather tool to provide accurate information."
    )
    .with_tool(get_weather)
    .with_memory_provider(oracle_memory_provider)
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o",
    })
    .build()
)

In [ ]:
weather_agent.save()

In [ ]:
# The agent will automatically use the tool when needed!
response = weather_agent.run("What's the weather like in New York? (latitude: 40.7128, longitude: -74.0060)")
print(f"\nAgent: {response}\n")

In [ ]:
# Ask follow-up questions
response2 = weather_agent.run("Is it warmer in Los Angeles? (latitude: 34.0522, longitude: -118.2437)")
print(f"Agent: {response2}\n")

# Part 6: Semantic Cache

In [ ]:
import time

embedding_config = {
    "model": "text-embedding-3-small",
    "api_key": os.getenv("OPENAI_API_KEY")  ,
}

# Build agent without cache
agent = (MemAgentBuilder()
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o-mini",
        "api_key":os.getenv("OPENAI_API_KEY"),
    })
    .with_memory_provider(oracle_memory_provider)
    .with_embedding_provider('openai', embedding_config)
    .build()
)

# Record time before query
start_time = time.time()

# Run without cache
response1 = agent.run("What's the capital of France?")
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
print(f"Agent: {response1}\n")

In [ ]:
# Now enable cache!
agent.enable_semantic_cache()

Run the cell below twice
- First run: No cache hit
- Second run: Returned cached response

In [ ]:
# These queries will use cache
start_time = time.time()
response2 = agent.run("What's the capital of France?") 
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
print(f"Agent: {response2}\n")

Run the cell below twice
- First run: No cache hit
- Second run: Returned cached response

In [ ]:
start_time = time.time()
response3 = agent.run("Tell me France's capital in small caps")
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
print(f"Agent: {response3}\n")

# Part 7: Summarization

In [ ]:
summary_ids = agent.generate_summaries(
    days_back=7,  # Look back 7 days (default)
    max_memories_per_summary=50  # Max memories per summary chunk (default)
)

In [ ]:
import time

embedding_config = {
    "model": "text-embedding-3-small",
    "api_key": os.getenv("OPENAI_API_KEY")  ,
}

# Build agent without cache
agent = (MemAgentBuilder()
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o-mini",
        "api_key":os.getenv("OPENAI_API_KEY"),
    })
    .with_memory_provider(oracle_memory_provider)
    .with_embedding_provider('openai', embedding_config)
    .build()
)

# Record time before query
start_time = time.time()

# Run without cache
response1 = agent.run("What's the capital of France?")
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
print(f"Agent: {response1}\n")